In [1]:
import numpy as np
import pandas as pd
import sklearn
import json
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from sklearn.model_selection import KFold 
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix, hstack, vstack
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.metrics import f1_score

## Importando DataSet

In [2]:
filename = 'USvideos.csv'
category_filename = 'US_category_id.json'

videos = pd.read_csv(filename)
videos = videos.dropna(axis=0)
print("DataSet null inputs: \n"+str(videos.isnull().sum())+"\n")
print(filename+" shape: "+str(videos.shape))
videos.head()

DataSet null inputs: 
video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
dtype: int64

USvideos.csv shape: (40379, 16)


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


# Tratamento do DataSet

## Correção de formato de data-hora para padrão Unix

In [3]:
videos['trending_date'] = pd.to_datetime(videos['trending_date'], format='%y.%d.%m')
videos['publish_time'] = pd.to_datetime(videos['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

# separates date and time into two columns from 'publish_time' column

videos.insert(4, 'publish_date', videos['publish_time'].dt.date)
videos['publish_time'] = videos['publish_time'].dt.time
videos['publish_date']=pd.to_datetime(videos['publish_date'])

videos.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,2017-11-13,22,17:13:01,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,2017-11-13,24,07:30:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,2017-11-12,23,19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,2017-11-13,24,11:00:04,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,2017-11-12,24,18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


## Tratando para pegar somente a ultima entrada para cada video (a entrada mais atualizada)

In [4]:
videos_lastentry = videos.sort_values('trending_date', ascending=False).drop_duplicates(['video_id'])
print(filename+" shape: "+str(videos_lastentry.shape))
videos_lastentry.head()

USvideos.csv shape: (6254, 17)


,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr..."
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...


## Buscando o nome da categoria pelo identificador e armazenando em nova coluna

In [5]:
with open(category_filename) as f:
    category = json.load(f)

In [6]:
def category_replace(c_id):
    for i in category["items"]:
        if int(c_id) == int(i["id"]):
             return i["snippet"]["title"]
    return "None"

In [7]:
videos_lastentry['category_name'] = videos_lastentry.apply(lambda row: category_replace(row['category_id']), axis=1)
print(filename+" shape: "+str(videos_lastentry.shape))
videos_lastentry.head()

USvideos.csv shape: (6254, 18)


,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music


## Transformando categorias em OneHotEncoder

In [8]:
# DESNECESSARIO - UTILIZADO ONEHOTENCODER() NO MAPPER
#videos_lastentry = videos_lastentry.join(pd.get_dummies(videos_lastentry['category_name']))
#videos_lastentry.head()

## Definindo sentimento por video

In [9]:
#positive sentiment = 60% da quantidade de likes+dislikes
videos_lastentry['sentiment'] = videos_lastentry['likes'] >= 0.6*(videos_lastentry['likes']+videos_lastentry['dislikes']) 
videos_lastentry.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming,True
40811,Nd3zqXro_P0,2018-06-14,Why 350°F is the magic number for baking,Vox,2018-06-07,25,12:00:02,"baking|""maillard reaction""|""s pen""|""Vox.com""|""...",799411,16092,652,1551,https://i.ytimg.com/vi/Nd3zqXro_P0/default.jpg,False,False,False,Turns out there’s a lot of chemistry in cookin...,News & Politics,True
40821,jWkOwC7vMb4,2018-06-14,“Bonding” with Grandma,itsAlexClark,2018-06-06,1,21:40:34,"its alex clark|""itsalexclark""|""alex clark""|""it...",1004181,48129,889,5483,https://i.ytimg.com/vi/jWkOwC7vMb4/default.jpg,False,False,False,"You know my family, now you need to meet my gr...",Film & Animation,True
40820,T9UHD30MSGk,2018-06-14,Knife Expert Guesses Cheap vs. Expensive Knive...,Epicurious,2018-06-06,26,18:54:38,"best knife|""boning knife""|""cheap""|""cheap vs ex...",903651,11427,2305,2504,https://i.ytimg.com/vi/T9UHD30MSGk/default.jpg,False,False,False,Epicurious challenges knife expert Geoff Feder...,Howto & Style,True
40819,NB3gWkhLkxM,2018-06-14,The Internet - Come Over (Official Video),TheInternetVEVO,2018-06-06,10,14:00:02,"Columbia|""Come Over""|""R&B/Soul""|""The Internet""",1615678,92403,1856,4572,https://i.ytimg.com/vi/NB3gWkhLkxM/default.jpg,False,False,False,The Internet's Come Over is taken from the new...,Music,True


## Normalização das features numéricas

In [10]:
videos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count
count,6254.000000,6.254000e+03,6.254000e+03,6.254000e+03,6.254000e+03
mean,20.408539,1.975222e+06,5.592469e+04,2.828081e+03,6.289861e+03
std,7.256811,7.092548e+06,1.942315e+05,2.433277e+04,2.943713e+04
min,1.000000,5.590000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,1.648875e+05,2.931000e+03,1.300000e+02,3.870000e+02
50%,24.000000,5.270375e+05,1.222400e+04,4.485000e+02,1.287500e+03
75%,25.000000,1.490734e+06,3.898500e+04,1.516000e+03,4.110750e+03
max,43.000000,2.252119e+08,5.613827e+06,1.643059e+06,1.228655e+06


In [11]:
# Normalização
videos_lastentry['views_nonnormal'] = videos_lastentry['views']
videos_lastentry['views'] = (videos_lastentry['views']-videos_lastentry['views'].min())/(videos_lastentry['views'].max()-videos_lastentry['views'].min())
#videos_lastentry['likes'] = (videos_lastentry['likes']-videos_lastentry['likes'].min())/(videos_lastentry['likes'].max()-videos_lastentry['likes'].min())
#videos_lastentry['dislikes'] = (videos_lastentry['dislikes']-videos_lastentry['dislikes'].min())/(videos_lastentry['dislikes'].max()-videos_lastentry['dislikes'].min())
videos_lastentry['comment_count'] = (videos_lastentry['comment_count']-videos_lastentry['comment_count'].min())/(videos_lastentry['comment_count'].max()-videos_lastentry['comment_count'].min())
videos_lastentry.describe()

,category_id,views,likes,dislikes,comment_count,views_nonnormal
count,6254.000000,6254.000000,6.254000e+03,6.254000e+03,6254.000000,6.254000e+03
mean,20.408539,0.008768,5.592469e+04,2.828081e+03,0.005119,1.975222e+06
std,7.256811,0.031493,1.942315e+05,2.433277e+04,0.023959,7.092548e+06
min,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,5.590000e+02
25%,17.000000,0.000730,2.931000e+03,1.300000e+02,0.000315,1.648875e+05
50%,24.000000,0.002338,1.222400e+04,4.485000e+02,0.001048,5.270375e+05
75%,25.000000,0.006617,3.898500e+04,1.516000e+03,0.003346,1.490734e+06
max,43.000000,1.000000,5.613827e+06,1.643059e+06,1.000000,2.252119e+08


## Separando conjuntos de treino, validação e teste

In [12]:
train_data = videos_lastentry.sample(frac=0.8, random_state=200)
test_data = videos_lastentry.drop(train_data.index)
print(train_data.shape, test_data.shape)

(5003, 20) (1251, 20)


# Classificador

## União de features

In [13]:
mapper = DataFrameMapper([
    ('title', CountVectorizer()),
    ('description', CountVectorizer(stop_words='english', max_features = 5000, strip_accents='ascii')),
    ('tags', CountVectorizer()),
    ('category_name', LabelBinarizer()),
    ('comments_disabled', None),
    ('views', None),
    ('comment_count', None)
])
x_train = mapper.fit_transform(train_data)
y_train = train_data['sentiment'].values

In [14]:
x_train.shape

(5003, 34951)

In [15]:
classifier = LogisticRegression(penalty = 'l1')
classifier.fit(x_train, y_train)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
x_test = mapper.transform(test_data)
y_test = test_data['sentiment'].values
predicted = classifier.predict(x_test)

In [17]:
probs = classifier.predict_proba(x_test)
print(probs)

[[9.52619973e-05 9.99904738e-01]
 [6.15853650e-03 9.93841464e-01]
 [4.56944400e-04 9.99543056e-01]
 ...
 [1.94093694e-04 9.99805906e-01]
 [4.59389081e-03 9.95406109e-01]
 [1.33964093e-03 9.98660359e-01]]


In [18]:
print("Acuracia: "+str(metrics.accuracy_score(y_test, predicted)))
print("ROC AUC score: "+str(metrics.roc_auc_score(y_test, probs[:, 1])))

Acuracia: 0.9664268585131894
ROC AUC score: 0.8310409163882502


In [19]:
predicted = classifier.predict(mapper.transform(videos_lastentry))

In [20]:
videos_output = videos_lastentry
videos_output['predicted_sentiment'] = predicted

videos_output = videos_output.sort_values('predicted_sentiment', ascending=False)
videos_output.head()

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,views_nonnormal,predicted_sentiment
40948,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,2018-05-17,20,17:09:38,"call of duty|""cod""|""activision""|""Black Ops 4""",0.045760,357079,...,0.117848,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,Gaming,True,10306119,True
10595,S4vKQbpOn5k,2018-01-05,The Applejack Problem,Jenny Nicholson,2017-12-27,1,04:42:08,[none],0.000480,5878,...,0.001327,https://i.ytimg.com/vi/S4vKQbpOn5k/default.jpg,False,False,False,Next video is about star wars I promise.\n\nCl...,Film & Animation,True,108662,True
10781,1oFQ76ctfS0,2018-01-06,"When it's 5 a.m. at an Irish Wedding - Ho, Ro,...",Max Skipper,2017-12-29,22,11:18:21,"irish weeding|""irish""|""clodagh mccarthy""|""ho r...",0.001651,8738,...,0.000603,https://i.ytimg.com/vi/1oFQ76ctfS0/default.jpg,False,False,False,Credit: Clodagh McCarthy\n\nFacebook: https://...,People & Blogs,True,372465,True
10782,q_MmN6-KLXg,2018-01-06,Chloe x Halle - Grown (Official Music Video),Chloe x Halle,2017-12-29,10,16:02:53,"Chloe x Halle|""Chloe and Halle""|""Parkwood Ente...",0.002902,19555,...,0.001111,https://i.ytimg.com/vi/q_MmN6-KLXg/default.jpg,False,False,False,"Official video for Grown, Watch Grown-ish on F...",Music,True,654210,True
10783,ejlD0Sg-8ig,2018-01-06,THIS HAS TO STOP! INSTAGRAM TRENDS I HATED IN ...,Patricia Bright,2017-12-29,26,22:11:30,"online fashion|""best online fashion shops""|""Fe...",0.004461,58618,...,0.004881,https://i.ytimg.com/vi/ejlD0Sg-8ig/default.jpg,False,False,False,So people its about to be a new year and I am ...,Howto & Style,True,1005215,True


In [21]:
f1_score(videos_output['sentiment'].astype(int), videos_output['predicted_sentiment'].astype(int), average='macro')

0.8549643126073102

In [22]:
get_n_features = 10
print("The "+str(get_n_features)+" most positive-weighted words are: ")
for feature_id in sorted(range(len(classifier.coef_[0])), key=lambda i: classifier.coef_[0][i], reverse=True)[:get_n_features]:
    print("%s (%f)" % (mapper.transformed_names_[feature_id], classifier.coef_[0][feature_id]))
print()
print("The "+str(get_n_features)+" most negative-weighted words are: ")
for feature_id in sorted(range(len(classifier.coef_[0])), key=lambda i: classifier.coef_[0][i], reverse=False)[:get_n_features]:
    print("%s (%f)" % (mapper.transformed_names_[feature_id], classifier.coef_[0][feature_id]))

The 10 most positive-weighted words are: 
category_name_Music (1.504686)
tags_2017 (0.859617)
tags_south (0.852206)
category_name_Howto & Style (0.850850)
tags_interview (0.850688)
description_allegations (0.742791)
category_name_Comedy (0.704401)
description_help (0.683364)
description_miller (0.647681)
description_org (0.601863)

The 10 most negative-weighted words are: 
title_metoo (-2.773094)
description_scoop (-2.484765)
title_clinton (-2.444337)
description_agent (-2.317480)
tags_carter (-2.018629)
title_trump (-1.962764)
tags_animatronic (-1.926358)
title_net (-1.917487)
tags_persil (-1.729394)
description_nash (-1.720043)


# K-Fold

In [23]:
def run_kfold(clf,X,Y):
    kf = KFold(n_splits=10)
    outcomes = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

In [26]:
run_kfold(classifier,x_train,y_train)

MemoryError: 

In [27]:
videos_lastentry['like_ratio'] = videos_lastentry['likes']/(videos_lastentry['likes']+videos_lastentry['dislikes'])

In [28]:
from IPython.display import HTML, display

# We choose the 10 most trending videos
selected_columns = ['title', 'channel_title', 'thumbnail_link', 'publish_date', 'category_name', 'likes', 'dislikes', 'views_nonnormal', 'like_ratio']

most_frequent = videos_lastentry.groupby(selected_columns)['video_id'].agg(
    {"code_count": len}).sort_values(by=['like_ratio']
).head(10).reset_index()

# Construction of HTML table with miniature photos assigned to the most popular movies
table_content = ''
max_title_length = 50

for date, row in most_frequent.T.iteritems():
    HTML_row = '<tr>'
    HTML_row += '<td><img src="' + str(row[2]) + '"style="width:100px;height:100px;"></td>'
    HTML_row += '<td>' + str(row[1]) + '</td>'
    HTML_row += '<td>' + str(row[0])  + '</td>'
    HTML_row += '<td>' + str(row[4]) + '</td>'
    HTML_row += '<td>' + str(row[3]) + '</td>'
    HTML_row += '<td>' + str(row[5]) + '</td>'
    HTML_row += '<td>' + str(row[6]) + '</td>'
    HTML_row += '<td>' + str(row[7]) + '</td>'
    
    table_content += HTML_row + '</tr>'

display(HTML(
    '<table><tr><th>Photo</th><th>Channel Name</th><th style="width:250px;">Title</th><th>Category</th><th>Publish Date</th><th>Likes</th><th>Dislikes</th><th>Views</th></tr>{}</table>'.format(table_content))
)

c:\users\work\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


Photo,Channel Name,Title,Category,Publish Date,Likes,Dislikes,Views
,Rob Andretti,Kelly Oubre Punches John Wall in the Lead during warriors wizards scuffle,Sports,2017-10-28 00:00:00,0,3,2447
,Daily Caller,PSA from Chairman of the FCC Ajit Pai,People & Blogs,2017-12-13 00:00:00,9572,228426,1205682
,Washington Post,The FCC repeals its net neutrality rules,News & Politics,2017-12-14 00:00:00,5930,132439,1263104
,J S,LuLaRoe Consultant Mocks Those With Special Needs,People & Blogs,2018-01-25 00:00:00,30,629,229332
,Roy Moore for Senate,Judge Roy Moore Campaign Statement,People & Blogs,2017-12-14 00:00:00,4329,59157,789818
,theSkimm,"Michael Wolff, “Fire and Fury | theSkimm Sip 'n Skimm",People & Blogs,2018-01-31 00:00:00,2,16,12989
,NBC News,Roy Moore Delivers Concession Speech From Alabama (Full) | NBC News,News & Politics,2017-12-13 00:00:00,625,4786,201025
,Ross Kempsell,Peter Bone to Trump: Delete Your Account,Comedy,2017-11-30 00:00:00,7,39,3052
,CBS News,Grammys 2018: Hillary Clinton surprises crowd to read Fire and Fury,News & Politics,2018-01-29 00:00:00,4028,20385,467531
,CBS Los Angeles,Jeffrey Tambor Fired From 'Transparent' Following Amazon Investigation Into Sexual Misconduct Allega,News & Politics,2018-02-16 00:00:00,3,15,1464


In [30]:
videos_lastentry.loc[videos_lastentry['tags'].str.contains("carter")]

,video_id,trending_date,title,channel_title,publish_date,category_id,publish_time,tags,views,likes,...,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,sentiment,views_nonnormal,predicted_sentiment,like_ratio
34740,PGRcxXLfWrM,2018-05-14,SCUBA DIVING IN POND FOR TREASURE!! (I FOUND IT!),Carter Sharer,2018-04-15,24,15:37:52,"#treasure|""scuba diving for treasure""|""finding...",0.028485,111486,...,https://i.ytimg.com/vi/PGRcxXLfWrM/default.jpg,False,False,False,I WENT SCUBA DIVING IN A POND LOOKING FOR BURI...,Entertainment,True,6415756,True,0.873585
31737,7_FJUSBFbJM,2018-04-29,Game of Zones - Game of Zones - S5:E1: 'A Gold...,Bleacher Report,2018-04-12,43,21:00:12,"bleacher report|""br""|""nba""|""game of zones""|""go...",0.006418,33261,...,https://i.ytimg.com/vi/7_FJUSBFbJM/default.jpg,False,False,False,"In the season 5 premiere of Game of Zones, the...",Shows,True,1445949,True,0.985920
18984,gtprpculjrQ,2018-02-19,Donovan Mitchell Wins 2018 Verizon Slam Dunk C...,NBA,2018-02-18,17,03:54:19,"nba|""highlights""|""basketball""|""plays""|""amazing...",0.001587,2378,...,https://i.ytimg.com/vi/gtprpculjrQ/default.jpg,False,False,False,Donovan Mitchell is your 2018 Verizon Slam Dun...,Sports,True,357946,True,0.873943
16970,GClwSu4hKpQ,2018-02-08,Shopping For Jugs | Growing Up Hip Hop: Atlant...,WE tv,2018-02-01,24,21:30:31,"WE tv|""Growing Up Hip Hop: ATL""|""GUHHATL""|""GUH...",0.000134,142,...,https://i.ytimg.com/vi/GClwSu4hKpQ/default.jpg,False,False,False,Bow Wow shows the girls the ropes at the strip...,Entertainment,False,30803,False,0.290984
15774,CxGUmtRLm5g,2018-02-02,Budweiser 2018 Super Bowl Commercial | “Stand ...,Budweiser,2018-01-26,24,13:23:59,"budweiser|""stand by you""|""super bowl""|""water""|...",0.041265,20467,...,https://i.ytimg.com/vi/CxGUmtRLm5g/default.jpg,False,False,False,"Since 1988, our employees have helped provide ...",Entertainment,True,9293828,True,0.803667
11196,PjqKPHZJgF0,2018-01-08,Lil Wayne - Family Feud feat. Drake (Official ...,Lil Wayne,2017-12-29,10,22:59:01,"lil wayne|""weezy""|""weezy wednesday""|""wayne car...",0.029213,133903,...,https://i.ytimg.com/vi/PjqKPHZJgF0/default.jpg,False,False,False,"Family Feud by Lil Wayne feat. Drake, off the ...",Music,True,6579609,True,0.947664
